In [1]:
# Interactive 3D scatter with Y = COUNT (within-age-bin rank), color = Gender.
# - X: Age (years)
# - Y: Population count per age bin (1-year bins), shown as 1..N stack
# - Z: Net worth (kEUR)
# - Color: Gender
# Requires: numpy, pandas, plotly

import numpy as np
import pandas as pd
import plotly.express as px

# ---------- helpers ----------
def norm_ppf(p):
    """Acklam inverse normal CDF approximation."""
    a = [-3.969683028665376e+01, 2.209460984245205e+02, -2.759285104469687e+02,
         1.383577518672690e+02, -3.066479806614716e+01, 2.506628277459239e+00]
    b = [-5.447609879822406e+01, 1.615858368580409e+02, -1.556989798598866e+02,
         6.680131188771972e+01, -1.328068155288572e+01]
    c = [-7.784894002430293e-03, -3.223964580411365e-01, -2.400758277161838e+00,
         -2.549732539343734e+00, 4.374664141464968e+00, 2.938163982698783e+00]
    d = [7.784695709041462e-03, 3.224671290700398e-01, 2.445134137142996e+00, 3.754408661907416e+00]
    plow, phigh = 0.02425, 1 - 0.02425
    p = np.asarray(p)
    out = np.empty_like(p, dtype=float)

    mask_low = p < plow
    mask_high = p > phigh
    mask_mid = (~mask_low) & (~mask_high)

    q = np.sqrt(-2*np.log(p[mask_low]))
    out[mask_low] = (((((c[0]*q + c[1])*q + c[2])*q + c[3])*q + c[4])*q + c[5]) / \
                    ((((d[0]*q + d[1])*q + d[2])*q + d[3])*q + 1)

    q = np.sqrt(-2*np.log(1 - p[mask_high]))
    out[mask_high] = -(((((c[0]*q + c[1])*q + c[2])*q + c[3])*q + c[4])*q + c[5]) / \
                      ((((d[0]*q + d[1])*q + d[2])*q + d[3])*q + 1)

    q = p[mask_mid] - 0.5
    r = q*q
    out[mask_mid] = (((((a[0]*r + a[1])*r + a[2])*r + a[3])*r + a[4])*r + a[5])*q / \
                    (((((b[0]*r + b[1])*r + b[2])*r + b[3])*r + b[4])*r + 1)
    return out

def median_networth_by_age(age):
    L, k, x0 = 300_000.0, 0.07, 48.0
    base = L / (1 + np.exp(-k * (age - x0)))
    decline = 1.0 - 0.015 * np.clip(age - 72.0, 0, None)
    decline = np.clip(decline, 0.6, 1.0)
    return base * decline

def sigma_by_age(age):
    return 0.7 + 0.5 * np.exp(-((age - 50.0) / 12.0)**2)

def debt_bulge(age):
    return 30_000.0 * np.exp(-((age - 30.0) / 8.0)**2) + 10_000.0 * np.exp(-((age - 38.0) / 10.0)**2)

# ---------- synthetic population ----------
rng = np.random.default_rng(11)
N = 1_000
AGE_MIN, AGE_MAX = 18.0, 90.0
BIN_WIDTH = 1.0       # years for the count axis stacking
JITTER_Y = 0.15       # jitter so stacks aren't single lines

# Age distribution (beta-shaped) on [18, 90]
a_shape, b_shape = 2.2, 2.6
ages = AGE_MIN + (AGE_MAX - AGE_MIN) * rng.beta(a_shape, b_shape, size=N)

# Gender
gender = rng.choice(["Female", "Male"], size=N, p=[0.5, 0.5])

# Latent percentile to sample wealth quantiles (not plotted)
latent_percentile = rng.uniform(0.005, 0.995, size=N)

# Wealth draw (lognormal quantile by age) + youth debt bulge
med = median_networth_by_age(ages)
sig = sigma_by_age(ages)
mu = np.log(np.maximum(med, 1.0))
wealth = np.exp(mu + sig * norm_ppf(latent_percentile)) - debt_bulge(ages)
wealth += rng.normal(0, 2000.0, size=N)  # idiosyncratic noise

df = pd.DataFrame({
    "Age": ages,
    "Gender": gender,
    "NetWorth_kEUR": wealth / 1000.0
})

# ---------- build Y = COUNT per age bin ----------
age_bins = np.floor((df["Age"] - AGE_MIN) / BIN_WIDTH).astype(int)
df["AgeBin"] = (AGE_MIN + age_bins * BIN_WIDTH).astype(int)  # left edge integer year
df["BinCount"] = df.groupby("AgeBin")["Age"].transform("size")

# Shuffle within each bin and assign rank 1..BinCount as Y coordinate
df = df.sample(frac=1.0, random_state=123).reset_index(drop=True)
df["Y_Count"] = df.groupby("AgeBin").cumcount() + 1
df["Y_Count"] = df["Y_Count"] + rng.uniform(-JITTER_Y, JITTER_Y, size=len(df))  # optional

# ---------- interactive 3D scatter ----------
fig = px.scatter_3d(
    df,
    x="Age",
    y="Y_Count",
    z="NetWorth_kEUR",
    color="Gender",
    opacity=0.65,
    title="Population Wealth — 3D Interactive Scatter (Y = population count per age bin)",
    hover_data={
        "Age":":.1f",
        "Y_Count":":.0f",
        "NetWorth_kEUR":":.1f",
        "Gender":True,
        "BinCount":True,
        "AgeBin":True
    }
)

fig.update_layout(
    scene=dict(
        xaxis_title="Age (years)",
        yaxis_title=f"Population count per age ({int(BIN_WIDTH)}-year bins)",
        zaxis_title="Net worth (kEUR)"
    )
)

# Enable scroll-zoom; hide the Plotly logo
#fig.show(config={"scrollZoom": True, "displaylogo": False, "responsive": True})

# Optional: save to standalone HTML for sharing
with open("wealth_scatter_3d_gender_Ycount.html", "w", encoding="utf-8") as f:
    f.write(fig.to_html(full_html=True, include_plotlyjs="cdn",
                        config={"scrollZoom": True, "displaylogo": False, "responsive": True}))

In [5]:
# ================================================================
# Population Wealth — 3D PAWNS (reliable + visible)
# X = Age | Y = COUNT per age (1-yr bins) | Z = Net worth (kEUR)
# Marker shape = chess pawn made with Mesh3d. Color by Gender.
# This version:
#  - fixes any empty/hidden-mesh issues,
#  - scales each pawn ANISOTROPICALLY so it’s clearly visible
#    relative to each axis span (no skinny needle effect),
#  - uses a sane camera + aspect so the cloud isn’t a vertical sliver.
# ================================================================

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

try:
    pio.renderers.default = "vscode"
except Exception:
    pio.renderers.default = "notebook_connected"

# -----------------------------
# Tunables
# -----------------------------
N = 10_000
AGE_MIN, AGE_MAX = 18.0, 90.0
BIN_WIDTH = 1.0
JITTER_Y = 0.15

# number of pawns per gender (meshes are heavy)
M_PER_GENDER = 150

# Pawn visual size controls
SIZE_MODE   = "fixed"        # "fixed" or "by_column"
# --- TUNABLES (top of the cell) ---
PAWN_SIZE_F = 0.035       # overall pawn size (fraction of axis span)
PAWN_Z_MULT = 2.5         # <<< make pawns TALLER by increasing this (e.g., 3.5 or 4.0)
HEAD_RADIUS = 0.24        # optional: bigger head helps the silhouette         # base fraction of each axis span used for pawn size
SIZE_COLUMN = "NetWorth_kEUR"
SIZE_MULT_RANGE = (0.7, 1.6) # if by_column, scale multipliers (min..max) after robust normalization

SEED = 7

# -----------------------------
# Synthetic pop (same idea)
# -----------------------------
# ---- DROP-IN REPLACEMENT for norm_ppf (no syntax errors) ----
# Paste this over your existing norm_ppf().

import numpy as np

def norm_ppf(p):
    """
    Peter J. Acklam's inverse normal CDF approximation.
    Vectorized, numerically safe, and with correctly balanced parentheses.
    """
    p = np.asarray(p, dtype=float)
    # keep away from 0 and 1 to avoid log(0)
    eps = np.finfo(float).tiny
    p = np.clip(p, eps, 1.0 - eps)

    a = [-3.969683028665376e+01,  2.209460984245205e+02,
         -2.759285104469687e+02,  1.383577518672690e+02,
         -3.066479806614716e+01,  2.506628277459239e+00]
    b = [-5.447609879822406e+01,  1.615858368580409e+02,
         -1.556989798598866e+02,  6.680131188771972e+01,
         -1.328068155288572e+01]
    c = [-7.784894002430293e-03, -3.223964580411365e-01,
         -2.400758277161838e+00, -2.549732539343734e+00,
          4.374664141464968e+00,  2.938163982698783e+00]
    d = [ 7.784695709041462e-03,  3.224671290700398e-01,
          2.445134137142996e+00,  3.754408661907416e+00]

    plow  = 0.02425
    phigh = 1.0 - plow

    x = np.empty_like(p)

    # lower region
    mask_low = p < plow
    if np.any(mask_low):
        q  = np.sqrt(-2.0 * np.log(p[mask_low]))
        num = (((((c[0]*q + c[1])*q + c[2])*q + c[3])*q + c[4])*q + c[5])
        den = ((((d[0]*q + d[1])*q + d[2])*q + d[3]) * q + 1.0)
        x[mask_low] = num / den

    # upper region
    mask_high = p > phigh
    if np.any(mask_high):
        q  = np.sqrt(-2.0 * np.log(1.0 - p[mask_high]))
        num = (((((c[0]*q + c[1])*q + c[2])*q + c[3])*q + c[4])*q + c[5])
        den = ((((d[0]*q + d[1])*q + d[2])*q + d[3]) * q + 1.0)
        x[mask_high] = -num / den

    # central region
    mask_mid = (~mask_low) & (~mask_high)
    if np.any(mask_mid):
        q = p[mask_mid] - 0.5
        r = q*q
        num = (((((a[0]*r + a[1])*r + a[2])*r + a[3])*r + a[4])*r + a[5]) * q
        den = (((((b[0]*r + b[1])*r + b[2])*r + b[3])*r + b[4])*r + 1.0)
        x[mask_mid] = num / den

    return x

def median_networth_by_age(age):
    L, k, x0 = 300_000.0, 0.07, 48.0
    base = L / (1 + np.exp(-k * (age - x0)))
    decline = 1.0 - 0.015 * np.clip(age - 72.0, 0, None)
    decline = np.clip(decline, 0.6, 1.0)
    return base * decline

def sigma_by_age(age): return 0.7 + 0.5*np.exp(-((age-50.0)/12.0)**2)
def debt_bulge(age):   return 30_000*np.exp(-((age-30)/8.0)**2) + 10_000*np.exp(-((age-38)/10.0)**2)

def make_population(n=N, seed=SEED):
    rng = np.random.default_rng(seed)
    ages = AGE_MIN + (AGE_MAX-AGE_MIN)*rng.beta(2.2, 2.6, size=n)
    gender = rng.choice(["Female","Male"], size=n)
    p = rng.uniform(0.005, 0.995, size=n)
    mu = np.log(np.maximum(median_networth_by_age(ages), 1.0))
    sig = sigma_by_age(ages)
    wealth = np.exp(mu + sig*norm_ppf(p)) - debt_bulge(ages) + rng.normal(0, 2000, size=n)
    df = pd.DataFrame({"Age":ages,"Gender":gender,"NetWorth_kEUR":wealth/1000.0})
    return df, rng

def add_y_count(df, rng):
    bins = np.floor((df["Age"]-AGE_MIN)/BIN_WIDTH).astype(int)
    df = df.copy()
    df["AgeBin"] = (AGE_MIN + bins*BIN_WIDTH).astype(int)
    df["BinCount"] = df.groupby("AgeBin")["Age"].transform("size")
    df = df.sample(frac=1.0, random_state=123).reset_index(drop=True)
    df["Y_Count"] = df.groupby("AgeBin").cumcount() + 1
    df["Y_Count"] = df["Y_Count"] + rng.uniform(-JITTER_Y, JITTER_Y, size=len(df))
    return df

# -----------------------------
# Pawn mesh (unit pawn, z∈[0,1])
# -----------------------------
def lathe_mesh(r_profile, z_profile, n_theta=20):
    r = np.asarray(r_profile); z = np.asarray(z_profile)
    nt = n_theta; nz = len(z)
    theta = np.linspace(0, 2*np.pi, nt, endpoint=False)
    ct, st = np.cos(theta), np.sin(theta)
    X = (r[:,None]*ct[None,:]).reshape(-1)
    Y = (r[:,None]*st[None,:]).reshape(-1)
    Z = (z[:,None] + np.zeros((nz, nt))).reshape(-1)
    def vid(zi, ti): return zi*nt + (ti % nt)
    I= []; J=[]; K=[]
    for zi in range(nz-1):
        for ti in range(nt):
            a=vid(zi,ti); b=vid(zi,ti+1); c=vid(zi+1,ti); d=vid(zi+1,ti+1)
            I.extend([a,a]); J.extend([b,d]); K.extend([d,c])
    return X,Y,Z,np.array(I,int),np.array(J,int),np.array(K,int)

def sphere_mesh(radius=0.18, center=(0.0,0.0,1.18), n_theta=18, n_phi=10):
    nt=n_theta; npf=n_phi
    theta=np.linspace(0,2*np.pi,nt,endpoint=False)
    phi=np.linspace(0,np.pi,npf)
    X=[];Y=[];Z=[]
    for p in range(npf):
        sp=np.sin(phi[p]); cp=np.cos(phi[p])
        for t in range(nt):
            ct=np.cos(theta[t]); st=np.sin(theta[t])
            X.append(center[0] + radius*ct*sp)
            Y.append(center[1] + radius*st*sp)
            Z.append(center[2] + radius*cp)
    X=np.array(X); Y=np.array(Y); Z=np.array(Z)
    def vid(p,t): return p*nt + (t%nt)
    I=[];J=[];K=[]
    for p in range(npf-1):
        for t in range(nt):
            a=vid(p,t); b=vid(p,t+1); c=vid(p+1,t); d=vid(p+1,t+1)
            I.extend([a,a]); J.extend([b,d]); K.extend([d,c])
    return X,Y,Z,np.array(I,int),np.array(J,int),np.array(K,int)

# Pawn silhouette (coarse), densified
z_prof = np.array([0.00, 0.08, 0.16, 0.28, 0.40, 0.52, 0.64, 0.74, 0.84, 0.92, 1.00])
r_prof = np.array([0.60, 0.70, 0.62, 0.45, 0.38, 0.28, 0.24, 0.28, 0.36, 0.30, 0.20])
z_dense = np.linspace(0,1,32)
r_dense = np.interp(z_dense, z_prof, r_prof)

bx,by,bz,bi,bj,bk = lathe_mesh(r_dense, z_dense, n_theta=22)
sx,sy,sz,si,sj,sk = sphere_mesh(radius=0.18, center=(0.0,0.0,1.18), n_theta=18, n_phi=12)

ux = np.concatenate([bx,sx]); uy = np.concatenate([by,sy]); uz = np.concatenate([bz,sz])
offset=len(bx)
ui = np.concatenate([bi, si+offset]); uj = np.concatenate([bj, sj+offset]); uk = np.concatenate([bk, sk+offset])

# -----------------------------
# Build data
# -----------------------------
df, rng = make_population(N, SEED)
df = add_y_count(df, rng)

# axis spans (robust on Z)
x_span = df["Age"].max() - df["Age"].min()
y_span = df["Y_Count"].max() - df["Y_Count"].min()
z_span = np.percentile(df["NetWorth_kEUR"], 97) - np.percentile(df["NetWorth_kEUR"], 3)

# base anisotropic scales so pawns occupy ~PAWN_SIZE_F of each axis span
BASE_SX = PAWN_SIZE_F * x_span
BASE_SY = PAWN_SIZE_F * y_span
BASE_SZ = PAWN_SIZE_F * z_span

# choose pawns
male_df   = df[df["Gender"]=="Male"].sample(n=min(M_PER_GENDER, (df["Gender"]=="Male").sum()), random_state=1)
female_df = df[df["Gender"]=="Female"].sample(n=min(M_PER_GENDER, (df["Gender"]=="Female").sum()), random_state=1)

def size_multiplier(series):
    if SIZE_MODE == "fixed":
        return np.full(len(series), 1.0)
    v = series.to_numpy()
    vmin, vmax = np.percentile(v, [5,95]); vmax = vmax if vmax>vmin else vmin+1e-6
    scaled = (np.clip(v, vmin, vmax)-vmin)/(vmax-vmin)
    lo, hi = SIZE_MULT_RANGE
    return lo + scaled*(hi-lo)

def replicate_aniso(sub):
    mult = size_multiplier(sub[SIZE_COLUMN]) if SIZE_MODE!="fixed" else np.ones(len(sub))
    X=[];Y=[];Z=[]; I=[];J=[];K=[]
    vcount=0
    for (px,py,pz,mm) in zip(sub["Age"], sub["Y_Count"], sub["NetWorth_kEUR"], mult):
        sx = BASE_SX*mm; sy = BASE_SY*mm; sz = BASE_SZ*mm
        X.append(px + ux*sx)
        Y.append(py + uy*sy)
        Z.append(pz + uz*sz)
        I.append(ui + vcount); J.append(uj + vcount); K.append(uk + vcount)
        vcount += len(ux)
    return np.concatenate(X), np.concatenate(Y), np.concatenate(Z), np.concatenate(I), np.concatenate(J), np.concatenate(K)

mx,my,mz,mi,mj,mk = replicate_aniso(male_df)
fx,fy,fz,fi,fj,fk = replicate_aniso(female_df)

# -----------------------------
# Plot
# -----------------------------
fig = go.Figure()

fig.add_trace(go.Mesh3d(
    x=mx, y=my, z=mz, i=mi, j=mj, k=mk,
    name="Male (pawns)", opacity=0.98, flatshading=True,
    lighting=dict(ambient=0.5, diffuse=0.8, specular=0.15, roughness=0.8),
    showscale=False
))
fig.add_trace(go.Mesh3d(
    x=fx, y=fy, z=fz, i=fi, j=fj, k=fk,
    name="Female (pawns)", opacity=0.98, flatshading=True,
    lighting=dict(ambient=0.5, diffuse=0.8, specular=0.15, roughness=0.8),
    showscale=False
))

# Equalize aspect so the cloud isn’t a skinny tower
fig.update_layout(
    title="Population Wealth — 3D Pawns (X=Age, Y=Count, Z=Net worth)",
    scene=dict(
        xaxis_title="Age (years)",
        yaxis_title=f"Population count per age ({int(BIN_WIDTH)}-yr bins)",
        zaxis_title="Net worth (kEUR)",
        aspectmode="manual",
        aspectratio=dict(x=1, y=1, z=0.6)  # compress Z visually
    ),
    legend=dict(itemsizing="constant")
)

# A camera that shows the spread well
fig.update_layout(scene_camera=dict(eye=dict(x=1.6, y=1.8, z=1.2)))

#fig.show(config={"scrollZoom": True, "displaylogo": False, "responsive": True})

# Optional: save
# fig.write_html("wealth_pawns_aniso.html", include_plotlyjs="cdn")
# Save HTML (open if inline rendering is blocked)
with open("wealth_pawns_3d_fix.html", "w", encoding="utf-8") as f:
    f.write(fig.to_html(full_html=True, include_plotlyjs="cdn",
                        config={"scrollZoom": True, "displaylogo": False, "responsive": True}))
print("Saved to wealth_pawns_3d_fix.html")

Saved to wealth_pawns_3d_fix.html


In [3]:
# ================================================================
# Population Wealth — 3D PAWNS (reliable + visible, taller version)
# X = Age | Y = COUNT per age (1-yr bins) | Z = Net worth (kEUR)
# Marker shape = chess pawn (Mesh3d), color by Gender.
# Changes:
#  - PAWN_GEOM_Z_MULT stretches the pawn geometry itself along Z
#  - PAWN_Z_MULT boosts scene-level vertical scale
#  - MIN_PAWN_HEIGHT_Z enforces a minimum height in data units (kEUR)
# ================================================================

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

try:
    pio.renderers.default = "vscode"
except Exception:
    pio.renderers.default = "notebook_connected"

# -----------------------------
# Tunables
# -----------------------------
N = 10_000
AGE_MIN, AGE_MAX = 18.0, 90.0
BIN_WIDTH = 1.0
JITTER_Y = 0.15

# number of pawns per gender (meshes are heavy)
M_PER_GENDER = 150

# Pawn visual size controls
SIZE_MODE   = "fixed"         # "fixed" or "by_column"
PAWN_SIZE_F = 0.035           # overall pawn size (fraction of axis span)
PAWN_Z_MULT = 2.5             # scene-level vertical boost (try 2.5–4.0)
PAWN_GEOM_Z_MULT = 2.2        # <<< NEW: stretch pawn geometry along Z
MIN_PAWN_HEIGHT_Z = 25.0      # <<< NEW: minimum height in kEUR (data units)
HEAD_RADIUS = 0.24            # bigger head helps silhouette
SIZE_COLUMN = "NetWorth_kEUR"
SIZE_MULT_RANGE = (0.7, 1.6)  # if by_column, per-point scale range

SEED = 7

# -----------------------------
# Synthetic pop (same idea)
# -----------------------------
def norm_ppf(p):
    """Acklam inverse normal CDF (stable, vectorized)."""
    p = np.asarray(p, dtype=float)
    eps = np.finfo(float).tiny
    p = np.clip(p, eps, 1.0 - eps)

    a = [-3.969683028665376e+01,  2.209460984245205e+02,
         -2.759285104469687e+02,  1.383577518672690e+02,
         -3.066479806614716e+01,  2.506628277459239e+00]
    b = [-5.447609879822406e+01,  1.615858368580409e+02,
         -1.556989798598866e+02,  6.680131188771972e+01,
         -1.328068155288572e+01]
    c = [-7.784894002430293e-03, -3.223964580411365e-01,
         -2.400758277161838e+00, -2.549732539343734e+00,
          4.374664141464968e+00,  2.938163982698783e+00]
    d = [ 7.784695709041462e-03,  3.224671290700398e-01,
          2.445134137142996e+00,  3.754408661907416e+00]

    plow, phigh = 0.02425, 1.0 - 0.02425
    x = np.empty_like(p)

    low = p < plow
    if np.any(low):
        q = np.sqrt(-2.0*np.log(p[low]))
        num = (((((c[0]*q + c[1])*q + c[2])*q + c[3])*q + c[4])*q + c[5])
        den = ((((d[0]*q + d[1])*q + d[2])*q + d[3]) * q + 1.0)
        x[low] = num / den

    high = p > phigh
    if np.any(high):
        q = np.sqrt(-2.0*np.log(1.0 - p[high]))
        num = (((((c[0]*q + c[1])*q + c[2])*q + c[3])*q + c[4])*q + c[5])
        den = ((((d[0]*q + d[1])*q + d[2])*q + d[3]) * q + 1.0)
        x[high] = -num / den

    mid = (~low) & (~high)
    if np.any(mid):
        q = p[mid] - 0.5
        r = q*q
        num = (((((a[0]*r + a[1])*r + a[2])*r + a[3])*r + a[4])*r + a[5]) * q
        den = (((((b[0]*r + b[1])*r + b[2])*r + b[3])*r + b[4])*r + 1.0)
        x[mid] = num / den
    return x

def median_networth_by_age(age):
    L, k, x0 = 300_000.0, 0.07, 48.0
    base = L / (1 + np.exp(-k * (age - x0)))
    decline = 1.0 - 0.015 * np.clip(age - 72.0, 0, None)
    decline = np.clip(decline, 0.6, 1.0)
    return base * decline

def sigma_by_age(age): return 0.7 + 0.5*np.exp(-((age-50.0)/12.0)**2)
def debt_bulge(age):   return 30_000*np.exp(-((age-30)/8.0)**2) + 10_000*np.exp(-((age-38)/10.0)**2)

def make_population(n=N, seed=SEED):
    rng = np.random.default_rng(seed)
    ages = AGE_MIN + (AGE_MAX-AGE_MIN)*rng.beta(2.2, 2.6, size=n)
    gender = rng.choice(["Female","Male"], size=n)
    p = rng.uniform(0.005, 0.995, size=n)
    mu = np.log(np.maximum(median_networth_by_age(ages), 1.0))
    sig = sigma_by_age(ages)
    wealth = np.exp(mu + sig*norm_ppf(p)) - debt_bulge(ages) + rng.normal(0, 2000, size=n)
    df = pd.DataFrame({"Age":ages,"Gender":gender,"NetWorth_kEUR":wealth/1000.0})
    return df, rng

def add_y_count(df, rng):
    bins = np.floor((df["Age"]-AGE_MIN)/BIN_WIDTH).astype(int)
    df = df.copy()
    df["AgeBin"] = (AGE_MIN + bins*BIN_WIDTH).astype(int)
    df["BinCount"] = df.groupby("AgeBin")["Age"].transform("size")
    df = df.sample(frac=1.0, random_state=123).reset_index(drop=True)
    df["Y_Count"] = df.groupby("AgeBin").cumcount() + 1
    df["Y_Count"] = df["Y_Count"] + rng.uniform(-JITTER_Y, JITTER_Y, size=len(df))
    return df

# -----------------------------
# Pawn mesh (unit pawn, z∈[0,1])
# -----------------------------
def lathe_mesh(r_profile, z_profile, n_theta=20):
    r = np.asarray(r_profile); z = np.asarray(z_profile)
    nt = n_theta; nz = len(z)
    theta = np.linspace(0, 2*np.pi, nt, endpoint=False)
    ct, st = np.cos(theta), np.sin(theta)
    X = (r[:,None]*ct[None,:]).reshape(-1)
    Y = (r[:,None]*st[None,:]).reshape(-1)
    Z = (z[:,None] + np.zeros((nz, nt))).reshape(-1)
    def vid(zi, ti): return zi*nt + (ti % nt)
    I= []; J=[]; K=[]
    for zi in range(nz-1):
        for ti in range(nt):
            a=vid(zi,ti); b=vid(zi,ti+1); c=vid(zi+1,ti); d=vid(zi+1,ti+1)
            I.extend([a,a]); J.extend([b,d]); K.extend([d,c])
    return X,Y,Z,np.array(I,int),np.array(J,int),np.array(K,int)

def sphere_mesh(radius=HEAD_RADIUS, center=(0.0,0.0,1.18), n_theta=18, n_phi=10):
    nt=n_theta; npf=n_phi
    theta=np.linspace(0,2*np.pi,nt,endpoint=False)
    phi=np.linspace(0,np.pi,npf)
    X=[];Y=[];Z=[]
    for p in range(npf):
        sp=np.sin(phi[p]); cp=np.cos(phi[p])
        for t in range(nt):
            ct=np.cos(theta[t]); st=np.sin(theta[t])
            X.append(center[0] + radius*ct*sp)
            Y.append(center[1] + radius*st*sp)
            Z.append(center[2] + radius*cp)
    X=np.array(X); Y=np.array(Y); Z=np.array(Z)
    def vid(p,t): return p*nt + (t%nt)
    I=[];J=[];K=[]
    for p in range(npf-1):
        for t in range(nt):
            a=vid(p,t); b=vid(p,t+1); c=vid(p+1,t); d=vid(p+1,t+1)
            I.extend([a,a]); J.extend([b,d]); K.extend([d,c])
    return X,Y,Z,np.array(I,int),np.array(J,int),np.array(K,int)

# Pawn silhouette (coarse), densified
z_prof = np.array([0.00, 0.08, 0.16, 0.28, 0.40, 0.52, 0.64, 0.74, 0.84, 0.92, 1.00])
r_prof = np.array([0.60, 0.70, 0.62, 0.45, 0.38, 0.28, 0.24, 0.28, 0.36, 0.30, 0.20])
z_dense = np.linspace(0,1,32)
r_dense = np.interp(z_dense, z_prof, r_prof)

bx,by,bz,bi,bj,bk = lathe_mesh(r_dense, z_dense, n_theta=22)
sx,sy,sz,si,sj,sk = sphere_mesh(radius=HEAD_RADIUS, center=(0.0,0.0,1.18), n_theta=18, n_phi=12)

ux = np.concatenate([bx,sx]); uy = np.concatenate([by,sy]); uz = np.concatenate([bz,sz])
offset=len(bx)
ui = np.concatenate([bi, si+offset]); uj = np.concatenate([bj, sj+offset]); uk = np.concatenate([bk, sk+offset])

# --- TALLER GEOMETRY: stretch the unit model along Z ---
uz = uz * PAWN_GEOM_Z_MULT

# -----------------------------
# Build data
# -----------------------------
df, rng = make_population(N, SEED)
df = add_y_count(df, rng)

# axis spans (robust on Z)
x_span = df["Age"].max() - df["Age"].min()
y_span = df["Y_Count"].max() - df["Y_Count"].min()
z_span = np.percentile(df["NetWorth_kEUR"], 97) - np.percentile(df["NetWorth_kEUR"], 3)

# base anisotropic scales so pawns occupy ~PAWN_SIZE_F of each axis span
BASE_SX = PAWN_SIZE_F * x_span
BASE_SY = PAWN_SIZE_F * y_span
BASE_SZ = PAWN_SIZE_F * z_span * PAWN_Z_MULT   # scene-level vertical boost

# choose pawns
male_df   = df[df["Gender"]=="Male"].sample(n=min(M_PER_GENDER, (df["Gender"]=="Male").sum()), random_state=1)
female_df = df[df["Gender"]=="Female"].sample(n=min(M_PER_GENDER, (df["Gender"]=="Female").sum()), random_state=1)

def size_multiplier(series):
    if SIZE_MODE == "fixed":
        return np.full(len(series), 1.0)
    v = series.to_numpy()
    vmin, vmax = np.percentile(v, [5,95]); vmax = vmax if vmax>vmin else vmin+1e-6
    scaled = (np.clip(v, vmin, vmax)-vmin)/(vmax-vmin)
    lo, hi = SIZE_MULT_RANGE
    return lo + scaled*(hi-lo)

def replicate_aniso(sub):
    mult = size_multiplier(sub[SIZE_COLUMN]) if SIZE_MODE!="fixed" else np.ones(len(sub))
    X=[];Y=[];Z=[]; I=[];J=[];K=[]
    vcount=0
    for (px,py,pz,mm) in zip(sub["Age"], sub["Y_Count"], sub["NetWorth_kEUR"], mult):
        sx = BASE_SX*mm
        sy = BASE_SY*mm
        sz = BASE_SZ*mm
        # --- enforce minimum visible height in data units (kEUR) ---
        if sz < MIN_PAWN_HEIGHT_Z:
            sz = MIN_PAWN_HEIGHT_Z

        X.append(px + ux*sx)
        Y.append(py + uy*sy)
        Z.append(pz + uz*sz)
        I.append(ui + vcount); J.append(uj + vcount); K.append(uk + vcount)
        vcount += len(ux)
    return np.concatenate(X), np.concatenate(Y), np.concatenate(Z), np.concatenate(I), np.concatenate(J), np.concatenate(K)

mx,my,mz,mi,mj,mk = replicate_aniso(male_df)
fx,fy,fz,fi,fj,fk = replicate_aniso(female_df)

# -----------------------------
# Plot
# -----------------------------
fig = go.Figure()

fig.add_trace(go.Mesh3d(
    x=mx, y=my, z=mz, i=mi, j=mj, k=mk,
    name="Male (pawns)", opacity=0.98, flatshading=True,
    lighting=dict(ambient=0.5, diffuse=0.8, specular=0.15, roughness=0.8),
    showscale=False
))
fig.add_trace(go.Mesh3d(
    x=fx, y=fy, z=fz, i=fi, j=fj, k=fk,
    name="Female (pawns)", opacity=0.98, flatshading=True,
    lighting=dict(ambient=0.5, diffuse=0.8, specular=0.15, roughness=0.8),
    showscale=False
))

fig.update_layout(
    title="Population Wealth — 3D Pawns (X=Age, Y=Count, Z=Net worth)",
    scene=dict(
        xaxis_title="Age (years)",
        yaxis_title=f"Population count per age ({int(BIN_WIDTH)}-yr bins)",
        zaxis_title="Net worth (kEUR)",
        aspectmode="manual",
        aspectratio=dict(x=1, y=1, z=1.0)   # un-flatten the scene
    ),
    legend=dict(itemsizing="constant")
)

fig.update_layout(scene_camera=dict(eye=dict(x=1.6, y=1.8, z=1.2)))

# Show and save
#fig.show(config={"scrollZoom": True, "displaylogo": False, "responsive": True})
with open("wealth_pawns_3d_fix.html", "w", encoding="utf-8") as f:
    f.write(fig.to_html(full_html=True, include_plotlyjs="cdn",
                        config={"scrollZoom": True, "displaylogo": False, "responsive": True}))
print("Saved to wealth_pawns_3d_fix.html")

Saved to wealth_pawns_3d_fix.html


In [6]:
# ================================================================
# Population Wealth — 3D PAWNS (taller + more visible, full script)
# X = Age | Y = COUNT per age (1-yr bins) | Z = Net worth (kEUR)
# Marker shape = chess pawn (Mesh3d), color by Gender.
# Tweaks included:
#  - PAWN_OPACITY to control transparency
#  - M_PER_GENDER increased for denser scene
#  - PAWN_GEOM_Z_MULT stretches pawn geometry along Z (true height change)
#  - PAWN_Z_MULT boosts scene-level vertical scale
#  - MIN_PAWN_HEIGHT_Z ensures a minimum height in data units (kEUR)
# ================================================================

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# Choose a renderer that works well in VS Code/Jupyter
try:
    pio.renderers.default = "vscode"
except Exception:
    pio.renderers.default = "notebook_connected"

# -----------------------------
# Tunables
# -----------------------------
N = 10_000
AGE_MIN, AGE_MAX = 18.0, 90.0
BIN_WIDTH = 1.0
JITTER_Y = 0.15

# number of pawns per gender (meshes are heavy)
M_PER_GENDER = 250        # bumped up a bit

# Pawn visual size controls
SIZE_MODE   = "fixed"     # "fixed" or "by_column"
PAWN_SIZE_F = 0.035       # overall pawn size (fraction of axis span)
PAWN_Z_MULT = 2.5         # scene-level vertical boost (try 2.5–4.0)
PAWN_GEOM_Z_MULT = 2.2    # stretch pawn geometry along Z (true height)
MIN_PAWN_HEIGHT_Z = 25.0  # minimum height in kEUR (data units)
HEAD_RADIUS = 0.24        # bigger head helps silhouette
SIZE_COLUMN = "NetWorth_kEUR"
SIZE_MULT_RANGE = (0.7, 1.6)  # if by_column, per-point scale range

# Opacity for pawns (lower -> more transparent)
PAWN_OPACITY = 0.4
FEMALE_PAWN_COLOR = "#d62728"
MALE_PAWN_COLOR = "#1f77b4"

SEED = 7

# -----------------------------
# Synthetic population model
# -----------------------------
def norm_ppf(p):
    """Acklam inverse normal CDF (stable, vectorized)."""
    p = np.asarray(p, dtype=float)
    eps = np.finfo(float).tiny
    p = np.clip(p, eps, 1.0 - eps)

    a = [-3.969683028665376e+01,  2.209460984245205e+02,
         -2.759285104469687e+02,  1.383577518672690e+02,
         -3.066479806614716e+01,  2.506628277459239e+00]
    b = [-5.447609879822406e+01,  1.615858368580409e+02,
         -1.556989798598866e+02,  6.680131188771972e+01,
         -1.328068155288572e+01]
    c = [-7.784894002430293e-03, -3.223964580411365e-01,
         -2.400758277161838e+00, -2.549732539343734e+00,
          4.374664141464968e+00,  2.938163982698783e+00]
    d = [ 7.784695709041462e-03,  3.224671290700398e-01,
          2.445134137142996e+00,  3.754408661907416e+00]

    plow, phigh = 0.02425, 1.0 - 0.02425
    x = np.empty_like(p)

    low = p < plow
    if np.any(low):
        q = np.sqrt(-2.0*np.log(p[low]))
        num = (((((c[0]*q + c[1])*q + c[2])*q + c[3])*q + c[4])*q + c[5])
        den = ((((d[0]*q + d[1])*q + d[2])*q + d[3]) * q + 1.0)
        x[low] = num / den

    high = p > phigh
    if np.any(high):
        q = np.sqrt(-2.0*np.log(1.0 - p[high]))
        num = (((((c[0]*q + c[1])*q + c[2])*q + c[3])*q + c[4])*q + c[5])
        den = ((((d[0]*q + d[1])*q + d[2])*q + d[3]) * q + 1.0)
        x[high] = -num / den

    mid = (~low) & (~high)
    if np.any(mid):
        q = p[mid] - 0.5
        r = q*q
        num = (((((a[0]*r + a[1])*r + a[2])*r + a[3])*r + a[4])*r + a[5]) * q
        den = (((((b[0]*r + b[1])*r + b[2])*r + b[3])*r + b[4])*r + 1.0)
        x[mid] = num / den
    return x

def median_networth_by_age(age):
    L, k, x0 = 300_000.0, 0.07, 48.0
    base = L / (1 + np.exp(-k * (age - x0)))
    decline = 1.0 - 0.015 * np.clip(age - 72.0, 0, None)
    decline = np.clip(decline, 0.6, 1.0)
    return base * decline

def sigma_by_age(age): return 0.7 + 0.5*np.exp(-((age-50.0)/12.0)**2)
def debt_bulge(age):   return 30_000*np.exp(-((age-30)/8.0)**2) + 10_000*np.exp(-((age-38)/10.0)**2)

def make_population(n=N, seed=SEED):
    rng = np.random.default_rng(seed)
    ages = AGE_MIN + (AGE_MAX-AGE_MIN)*rng.beta(2.2, 2.6, size=n)
    gender = rng.choice(["Female","Male"], size=n)
    p = rng.uniform(0.005, 0.995, size=n)
    mu = np.log(np.maximum(median_networth_by_age(ages), 1.0))
    sig = sigma_by_age(ages)
    wealth = np.exp(mu + sig*norm_ppf(p)) - debt_bulge(ages) + rng.normal(0, 2000, size=n)
    df = pd.DataFrame({"Age":ages,"Gender":gender,"NetWorth_kEUR":wealth/1000.0})
    return df, rng

def add_y_count(df, rng):
    bins = np.floor((df["Age"]-AGE_MIN)/BIN_WIDTH).astype(int)
    df = df.copy()
    df["AgeBin"] = (AGE_MIN + bins*BIN_WIDTH).astype(int)
    df["BinCount"] = df.groupby("AgeBin")["Age"].transform("size")
    df = df.sample(frac=1.0, random_state=123).reset_index(drop=True)
    df["Y_Count"] = df.groupby("AgeBin").cumcount() + 1
    df["Y_Count"] = df["Y_Count"] + rng.uniform(-JITTER_Y, JITTER_Y, size=len(df))
    return df

# -----------------------------
# Pawn mesh (unit pawn, z∈[0,1])
# -----------------------------
def lathe_mesh(r_profile, z_profile, n_theta=20):
    """Revolve (r(z), z) around z-axis -> triangular mesh."""
    r = np.asarray(r_profile); z = np.asarray(z_profile)
    nt = n_theta; nz = len(z)
    theta = np.linspace(0, 2*np.pi, nt, endpoint=False)
    ct, st = np.cos(theta), np.sin(theta)
    X = (r[:,None]*ct[None,:]).reshape(-1)
    Y = (r[:,None]*st[None,:]).reshape(-1)
    Z = (z[:,None] + np.zeros((nz, nt))).reshape(-1)
    def vid(zi, ti): return zi*nt + (ti % nt)
    I= []; J=[]; K=[]
    for zi in range(nz-1):
        for ti in range(nt):
            a=vid(zi,ti); b=vid(zi,ti+1); c=vid(zi+1,ti); d=vid(zi+1,ti+1)
            I.extend([a,a]); J.extend([b,d]); K.extend([d,c])
    return X,Y,Z,np.array(I,int),np.array(J,int),np.array(K,int)

def sphere_mesh(radius, center=(0.0,0.0,1.18), n_theta=18, n_phi=10):
    """Low-res sphere (lat-long parameterization)."""
    nt=n_theta; npf=n_phi
    theta=np.linspace(0,2*np.pi,nt,endpoint=False)
    phi=np.linspace(0,np.pi,npf)
    X=[];Y=[];Z=[]
    for p in range(npf):
        sp=np.sin(phi[p]); cp=np.cos(phi[p])
        for t in range(nt):
            ct=np.cos(theta[t]); st=np.sin(theta[t])
            X.append(center[0] + radius*ct*sp)
            Y.append(center[1] + radius*st*sp)
            Z.append(center[2] + radius*cp)
    X=np.array(X); Y=np.array(Y); Z=np.array(Z)
    def vid(p,t): return p*nt + (t%nt)
    I=[];J=[];K=[]
    for p in range(npf-1):
        for t in range(nt):
            a=vid(p,t); b=vid(p,t+1); c=vid(p+1,t); d=vid(p+1,t+1)
            I.extend([a,a]); J.extend([b,d]); K.extend([d,c])
    return X,Y,Z,np.array(I,int),np.array(J,int),np.array(K,int)

# Pawn silhouette (coarse), densified
z_prof = np.array([0.00, 0.08, 0.16, 0.28, 0.40, 0.52, 0.64, 0.74, 0.84, 0.92, 1.00])
r_prof = np.array([0.60, 0.70, 0.62, 0.45, 0.38, 0.28, 0.24, 0.28, 0.36, 0.30, 0.20])
z_dense = np.linspace(0,1,32)
r_dense = np.interp(z_dense, z_prof, r_prof)

bx,by,bz,bi,bj,bk = lathe_mesh(r_dense, z_dense, n_theta=22)
sx,sy,sz,si,sj,sk = sphere_mesh(radius=HEAD_RADIUS, center=(0.0,0.0,1.18), n_theta=18, n_phi=12)

ux = np.concatenate([bx,sx]); uy = np.concatenate([by,sy]); uz = np.concatenate([bz,sz])
offset=len(bx)
ui = np.concatenate([bi, si+offset]); uj = np.concatenate([bj, sj+offset]); uk = np.concatenate([bk, sk+offset])

# --- stretch the unit pawn geometry vertically (true taller pawns) ---
uz = uz * PAWN_GEOM_Z_MULT

# -----------------------------
# Build data
# -----------------------------
df, rng = make_population(N, SEED)
df = add_y_count(df, rng)

# axis spans (robust on Z)
x_span = df["Age"].max() - df["Age"].min()
y_span = df["Y_Count"].max() - df["Y_Count"].min()
z_span = np.percentile(df["NetWorth_kEUR"], 97) - np.percentile(df["NetWorth_kEUR"], 3)

# base anisotropic scales so pawns occupy ~PAWN_SIZE_F of each axis span
BASE_SX = PAWN_SIZE_F * x_span
BASE_SY = PAWN_SIZE_F * y_span
BASE_SZ = PAWN_SIZE_F * z_span * PAWN_Z_MULT   # scene-level vertical boost

# choose pawns
male_df   = df[df["Gender"]=="Male"].sample(n=min(M_PER_GENDER, (df["Gender"]=="Male").sum()), random_state=1)
female_df = df[df["Gender"]=="Female"].sample(n=min(M_PER_GENDER, (df["Gender"]=="Female").sum()), random_state=1)

def size_multiplier(series):
    if SIZE_MODE == "fixed":
        return np.full(len(series), 1.0)
    v = series.to_numpy()
    vmin, vmax = np.percentile(v, [5,95]); vmax = vmax if vmax>vmin else vmin+1e-6
    scaled = (np.clip(v, vmin, vmax)-vmin)/(vmax-vmin)
    lo, hi = SIZE_MULT_RANGE
    return lo + scaled*(hi-lo)

def replicate_aniso(sub):
    """Replicate unit pawn mesh at each point with anisotropic scaling and min-height guard."""
    mult = size_multiplier(sub[SIZE_COLUMN]) if SIZE_MODE!="fixed" else np.ones(len(sub))
    X=[];Y=[];Z=[]; I=[];J=[];K=[]
    vcount=0
    for (px,py,pz,mm) in zip(sub["Age"], sub["Y_Count"], sub["NetWorth_kEUR"], mult):
        sx = BASE_SX*mm
        sy = BASE_SY*mm
        sz = BASE_SZ*mm
        # ensure minimum visible height in data units (kEUR)
        if sz < MIN_PAWN_HEIGHT_Z:
            sz = MIN_PAWN_HEIGHT_Z

        X.append(px + ux*sx)
        Y.append(py + uy*sy)
        Z.append(pz + uz*sz)
        I.append(ui + vcount); J.append(uj + vcount); K.append(uk + vcount)
        vcount += len(ux)
    return np.concatenate(X), np.concatenate(Y), np.concatenate(Z), np.concatenate(I), np.concatenate(J), np.concatenate(K)

mx,my,mz,mi,mj,mk = replicate_aniso(male_df)
fx,fy,fz,fi,fj,fk = replicate_aniso(female_df)

# -----------------------------
# Plot
# -----------------------------
fig = go.Figure()

fig.add_trace(go.Mesh3d(
    x=mx, y=my, z=mz, i=mi, j=mj, k=mk,
    name="Male (pawns)",
    opacity=PAWN_OPACITY,
    color=MALE_PAWN_COLOR,
    flatshading=True,
    lighting=dict(ambient=0.5, diffuse=0.8, specular=0.15, roughness=0.8),
    showscale=False
))
fig.add_trace(go.Mesh3d(
    x=fx, y=fy, z=fz, i=fi, j=fj, k=fk,
    name="Female (pawns)",
    opacity=PAWN_OPACITY,
    color=FEMALE_PAWN_COLOR,
    flatshading=True,
    lighting=dict(ambient=0.5, diffuse=0.8, specular=0.15, roughness=0.8),
    showscale=False
))

fig.update_layout(
    title="Population Wealth — 3D Pawns (X=Age, Y=Count, Z=Net worth)",
    scene=dict(
        xaxis_title="Age (years)",
        yaxis_title=f"Population count per age ({int(BIN_WIDTH)}-yr bins)",
        zaxis_title="Net worth (kEUR)",
        aspectmode="manual",
        aspectratio=dict(x=1, y=1, z=1.0)  # un-flatten the scene
    ),
    legend=dict(itemsizing="constant")
)
fig.update_layout(scene_camera=dict(eye=dict(x=1.6, y=1.8, z=1.2)))

#fig.show(config={"scrollZoom": True, "displaylogo": False, "responsive": True})

# Optional: save to HTML
with open("wealth_pawns_3d_full.html", "w", encoding="utf-8") as f:
    f.write(fig.to_html(full_html=True, include_plotlyjs="cdn",
                        config={"scrollZoom": True, "displaylogo": False, "responsive": True}))